#####  Si lo que queremos conseguir es una RNA que de un error en test lo menor posible, ¿por qué no se escoge la RNA correspondiente al ciclo con un menor error en test?

###### Se escoje el mejor error de validación porque este a diferencia del de test tiene en cuenta el sobreajuste y la generalizacion de la RNA

#####  Desarrollar una función llamada holdOut que dados dos parámetros, N (igual al número de patrones) y P (valor entre 0 y 1, que indica el porcentaje de patrones que se separarán para el conjunto de test), devuelva una tupla con dos vectores con los índices de los patrones que serán utilizados para entrenamiento y test. La suma de longitudes de ambos vectores tiene que ser igual a N, y estos dos vectores tienen que ser disjuntos. 

###### * Esta función se puede hacer en de una forma muy sencilla usando la función randperm (para usarla, cargar el módulo con using Random) 

###### A partir de esta función, partir una base de datos en dos subconjuntos se realiza de una forma inmediata. 
######  Para realizar esta función no se permite el uso de bucles. 

In [1]:
function holdOut(N::Int, P::Real)
    if P < 0 || P > 1
        error("P debe ser un valor entre 0 y 1")
    end
    if N < 1
        error("N debe ser un valor mayor a 0")
    end
    indices = randperm(N)
    n_test = round(Int, N*P)
    return indices[n_test+1:end], indices[1:n_test]
end

holdOut (generic function with 1 method)

##### Desarrollar otra función llamada holdOut, igual que en la anterior y que, basándose en ella, tome 3 parámetros: N (número de patrones), Pval (tasa de patrones en el conjunto de validación) y Ptest (tasa de patrones en el conjunto de test), y devuelva una tupla con 3 vectores, con los índices de los elementos de los conjuntos de entrenamiento, validación y test. La suma de las longitudes de estos 3 vectores tiene que ser igual a N. 
###### * Para realizar esto, simplemente hay que hacer dos llamadas a la función holdOut desarrollada previamente. 
###### Al igual que en la función anterior, para realizar esta no se permite el uso de bucles

In [2]:
function holdOut(N::Int, Pval::Real, Ptest::Real) 
    if Pval < 0 || Pval > 1 || Ptest < 0 || Ptest > 1
        error("Pval y Ptest deben ser valores entre 0 y 1")
    end
    if N < 1
        error("N debe ser un valor mayor a 0")
    end
    if Pval + Ptest > 1
        error("La suma de Pval y Ptest debe ser menor o igual a 1")
    end
    indices = randperm(N)
    n_val = round(Int, N*Pval)
    n_test = round(Int, N*Ptest)
    return indices[n_val+n_test+1:end], indices[n_val+1:n_val+n_test], indices[1:n_val]
end

holdOut (generic function with 2 methods)

 ##### Modificar la función trainClassANN de entrenamiento de la RNA realizada en la práctica anterior para que acepte los siguientes parámetros opcionales a mayores de los definidos en la práctica anterior: 

In [4]:
function trainClassANN(topology::AbstractArray{<:Int,1}, 
    trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}};
    validationDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}=
        (Array{eltype(trainingDataset[1]),2}(undef,0,0), falses(0,0)),
    testDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}=
        (Array{eltype(trainingDataset[1]),2}(undef,0,0), falses(0,0)),
    transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),
    maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01, 
    maxEpochsVal::Int=20)

inputs = trainingDataset[1]'
targets = trainingDataset[2]'
numInputs = size(inputs, 1)
numOutputs = size(targets, 1)
ann = buildClassANN(numInputs, topology, numOutputs; transferFunctions=transferFunctions)

loss(model, x, y) = (size(y,1) == 1) ? Losses.binarycrossentropy(model(x),y) : Losses.crossentropy(model(x),y)

opt_state = Flux.setup(Adam(learningRate), ann)

loss_vector_train = zeros(maxEpochs)
loss_vector_val = zeros(maxEpochs)
loss_vector_test = zeros(maxEpochs)

best_ann = deepcopy(ann)
best_loss_val = Inf
patience = 0

for epoch in 1:maxEpochs
Flux.train!(loss, ann, [(inputs, targets)], opt_state)

loss_vector_train[epoch] = loss(ann, inputs, targets)

if !isempty(validationDataset[1])
inputs_val = validationDataset[1]'
targets_val = validationDataset[2]'
loss_vector_val[epoch] = loss(ann, inputs_val, targets_val)

if loss_vector_val[epoch] < best_loss_val
best_loss_val = loss_vector_val[epoch]
best_ann = deepcopy(ann)
patience = 0
else
patience += 1
if patience >= maxEpochsVal
 break
end
end
end

if !isempty(testDataset[1])
inputs_test = testDataset[1]'
targets_test = testDataset[2]'
loss_vector_test[epoch] = loss(ann, inputs_test, targets_test)
end
end 

if isempty(validationDataset[1])
best_ann = deepcopy(ann)
end

return (best_ann, loss_vector_train[1:epoch], loss_vector_val[1:epoch], loss_vector_test[1:epoch])
end 


trainClassANN (generic function with 1 method)

##### Dado que había dos versiones de la función de entrenamiento de RR.NN.AA. (la segunda aceptaba como salidas deseadas un vector), modificar de la misma manera esta función para que las salidas deseadas sean vectores en lugar de matrices para el caso de clasificación en dos clases. Tened en cuenta que en este caso los tipos de los conjuntos de entrenamiento, validación y test pasados como parámetros serán Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}} para los tres y por tanto será necesario llamar a la función reshape en los tres. 

In [6]:
function trainClassANN(topology::AbstractArray{<:Int,1},
    (inputs, targets)::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}};
    validationDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}}=
        (Array{eltype(inputs),2}(undef,0,0), falses(0)),
    testDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}}=
        (Array{eltype(inputs),2}(undef,0,0), falses(0)),
    transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),
    maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01, 
    maxEpochsVal::Int=20) 

targets = reshape(targets, (size(targets, 2), 1))  # Redimensionar a vector
trainClassANN(topology, (inputs, targets); transferFunctions = transferFunctions, 
maxEpochs, minLoss, learningRate, maxEpochsVal)
end


trainClassANN (generic function with 2 methods)